In [1]:
import hyppo
import numpy as np
from numba import njit
from hyppo.independence.base import IndependenceTest
from hyppo._utils import perm_test
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import copy 
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.tree._classes import DecisionTreeClassifier
from joblib import Parallel, delayed
from scipy.stats import entropy, multivariate_normal
from scipy.integrate import nquad

In [8]:
from hyppo.sims import ksample_sim
from hyppo.ksample._utils import k_sample_transform
from hyppo.sims import *

In [2]:
import sys
sys.executable
sys.path
sys.path.append('C:\\Users\\siptest\\AppData\\Roaming\\Python\\Python36\\Scripts')

In [3]:
def uf(X, y, n_estimators = 300, max_samples = .4, base = np.exp(1), kappa = 3):
    
    # Build forest with default parameters.
    model = BaggingClassifier(DecisionTreeClassifier(), 
                              n_estimators=n_estimators, 
                              max_samples=max_samples, 
                              bootstrap=False)
    model.fit(X, y)
    n = X.shape[0]
    K = model.n_classes_
    _, y = np.unique(y, return_inverse=True)
    
    cond_entropy = 0
    for tree_idx, tree in enumerate(model):
        # Find the indices of the training set used for partition.
        sampled_indices = model.estimators_samples_[tree_idx]
        unsampled_indices = np.delete(np.arange(0,n), sampled_indices)
        
        # Randomly split the rest into voting and evaluation.
        total_unsampled = len(unsampled_indices)
        np.random.shuffle(unsampled_indices)
        vote_indices = unsampled_indices[:total_unsampled//2]
        eval_indices = unsampled_indices[total_unsampled//2:]
        
        # Store the posterior in a num_nodes-by-num_classes matrix.
        # Posteriors in non-leaf cells will be zero everywhere
        # and later changed to uniform.
        node_counts = tree.tree_.n_node_samples
        class_counts = np.zeros((len(node_counts), K))
        est_nodes = tree.apply(X[vote_indices])
        est_classes = y[vote_indices]
        for i in range(len(est_nodes)):
            class_counts[est_nodes[i], est_classes[i]] += 1
        
        row_sums = class_counts.sum(axis=1) # Total number of estimation points in each leaf.
        row_sums[row_sums == 0] = 1 # Avoid divide by zero.
        class_probs = class_counts / row_sums[:, None]
        
        # Make the nodes that have no estimation indices uniform.
        # This includes non-leaf nodes, but that will not affect the estimate.
        class_probs[np.argwhere(class_probs.sum(axis = 1) == 0)] = [1 / K]*K
        
        # Apply finite sample correction and renormalize.
        where_0 = np.argwhere(class_probs == 0)
        for elem in where_0:
            class_probs[elem[0], elem[1]] = 1 / (kappa*class_counts.sum(axis = 1)[elem[0]])
        row_sums = class_probs.sum(axis=1)
        class_probs = class_probs / row_sums[:, None]
        
        # Place evaluation points in their corresponding leaf node.
        # Store evaluation posterior in a num_eval-by-num_class matrix.
        eval_class_probs = class_probs[tree.apply(X[eval_indices])]
        # eval_class_probs = [class_probs[x] for x in tree.apply(X[eval_indices])]
        eval_entropies = [entropy(posterior) for posterior in eval_class_probs]
        cond_entropy += np.mean(eval_entropies)

      
    return cond_entropy / n_estimators

In [4]:
#n = 20 used for csv 1, 2
n = 50 
#n = 6000
mus = range(5)
ds = range(1, 16)
mu = 1
num_trials = 10
#reps = 1
d = 2
pis = [0.05 * i for i in range(1, 20)]

In [5]:
MAX_SAMPLE_SIZE = 100
#STEP_SIZE = 5
STEP_SIZE = 20
SAMP_SIZES = range(20, MAX_SAMPLE_SIZE + STEP_SIZE, STEP_SIZE)
#SAMP_SIZES = range(5, MAX_SAMPLE_SIZE + STEP_SIZE, STEP_SIZE)
POWER_REPS = 20

In [16]:
def test(): 
    nullDist = []
    success= 0
    for i in range(1000): 
        temp = []
        success = np.random.binomial(1000, 0.5)
        temp.append(success/1000)
        temp.append((1000-success)/1000)
        nullDist.append(entropy(temp, base = np.exp(1)))
    #print(nullDist)
    
    matrix1, matrix2 = ksample_sim.trans_2samp(multimodal_independence, 500, 1, noise=True, trans=0.3)
    x, y = k_sample_transform([matrix1, matrix2])
    ce = uf(x, y)
    #to calculate p-value, get proportion of null greater then ce from UF 
    pvalue = (nullDist <= ce).sum() / 1000
    
    print(nullDist, ce, pvalue) 
    return nullDist, ce, pvalue

In [17]:
power = []
for x in SAMP_SIZES:
    pvalues = []
    num = 0; 
    for p in range(POWER_REPS): 
        null_Dist, test_stat, p_val = test()
        #pvalue = (null_Dist <= test_stat).sum() / 1000
        pvalues.append(p_val)
    for pval in pvalues: 
        if pval >= 0.05: 
            num=num+1
    power.append(num/ POWER_REPS)

np.savetxt('C:/Users/siptest/Desktop/NDD/UFIndependenceBinomialNull.csv',
               power, delimiter=',')

C:\Users\siptest\anaconda3\envs\py36\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[0.6926971130356336, 0.6931151802186033, 0.6931391805386118, 0.6929471672244782, 0.6924250066981698, 0.6931291804519437, 0.6924990405193354, 0.6931391805386118, 0.6925690691470865, 0.6927551293225422, 0.6924990405193354, 0.6926350931428009, 0.6906951757946529, 0.6931291804519437, 0.6929051610348317, 0.6931391805386118, 0.6930491773583609, 0.6920888071224436, 0.692809142468311, 0.6930971797265786, 0.6917945705986355, 0.6923469670899615, 0.6920888071224436, 0.6930191750980527, 0.6924250066981698, 0.6929851718108113, 0.6931151802186033, 0.6930191750980527, 0.6925690691470865, 0.6929851718108113, 0.6926350931428009, 0.6930191750980527, 0.6931151802186033, 0.6929051610348317, 0.6931291804519437, 0.6929851718108113, 0.6930491773583609, 0.6929051610348317, 0.6929471672244782, 0.6930491773583609, 0.6930971797265786, 0.6931151802186033, 0.6923469670899615, 0.6929851718108113, 0.6930191750980527, 0.6924250066981698, 0.6926971130356336, 0.693145180558612, 0.6899437584583995, 0.6924250066981698, 0

C:\Users\siptest\anaconda3\envs\py36\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[0.6924250066981698, 0.6914642362473639, 0.6929471672244782, 0.6929851718108113, 0.6921788679764836, 0.6930751788318458, 0.6930191750980527, 0.6931151802186033, 0.6930971797265786, 0.6917945705986355, 0.6926350931428009, 0.6928591529055732, 0.6925690691470865, 0.6924990405193354, 0.6930971797265786, 0.6930971797265786, 0.6930191750980527, 0.6930191750980527, 0.6930971797265786, 0.6931391805386118, 0.6931151802186033, 0.6929851718108113, 0.6924990405193354, 0.6930751788318458, 0.6930491773583609, 0.6926971130356336, 0.6925690691470865, 0.6931291804519437, 0.6918966592050799, 0.6921788679764836, 0.6920888071224436, 0.6925690691470865, 0.693145180558612, 0.6929851718108113, 0.6930191750980527, 0.6930191750980527, 0.6930491773583609, 0.6931471805599453, 0.6918966592050799, 0.691994737783755, 0.6924990405193354, 0.690833395474842, 0.6930491773583609, 0.6930491773583609, 0.6926350931428009, 0.6931391805386118, 0.6924250066981698, 0.6926350931428009, 0.691994737783755, 0.693145180558612, 0.69

C:\Users\siptest\anaconda3\envs\py36\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

[0.6916884711441548, 0.691994737783755, 0.6921788679764836, 0.6926350931428009, 0.6926350931428009, 0.6928591529055732, 0.692809142468311, 0.6931471805599453, 0.6930491773583609, 0.6924990405193354, 0.6929051610348317, 0.6929851718108113, 0.6930971797265786, 0.6929851718108113, 0.6930491773583609, 0.6927551293225422, 0.6923469670899615, 0.6930971797265786, 0.6930751788318458, 0.6931291804519437, 0.6920888071224436, 0.6926350931428009, 0.6930191750980527, 0.6926350931428009, 0.6929471672244782, 0.6926350931428009, 0.691994737783755, 0.6926350931428009, 0.6931291804519437, 0.6930751788318458, 0.6904066761889504, 0.6931391805386118, 0.6926971130356336, 0.6931391805386118, 0.6920888071224436, 0.6931291804519437, 0.6929471672244782, 0.6915783599888489, 0.6931391805386118, 0.6926971130356336, 0.6930491773583609, 0.6929851718108113, 0.6912239473013626, 0.693145180558612, 0.6930971797265786, 0.6929851718108113, 0.693145180558612, 0.6912239473013626, 0.693145180558612, 0.6931391805386118, 0.693

C:\Users\siptest\anaconda3\envs\py36\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[0.6930971797265786, 0.6931391805386118, 0.6931291804519437, 0.6922649210688046, 0.6923469670899615, 0.6915783599888489, 0.692809142468311, 0.6931291804519437, 0.6924250066981698, 0.6929471672244782, 0.6909675965703669, 0.6918966592050799, 0.6928591529055732, 0.6929851718108113, 0.6930971797265786, 0.6930491773583609, 0.693145180558612, 0.6926971130356336, 0.6931291804519437, 0.6929851718108113, 0.6931151802186033, 0.6923469670899615, 0.6928591529055732, 0.6929471672244782, 0.6924250066981698, 0.693145180558612, 0.6918966592050799, 0.6929051610348317, 0.6931151802186033, 0.6924250066981698, 0.6915783599888489, 0.691994737783755, 0.6915783599888489, 0.6923469670899615, 0.6929851718108113, 0.6929471672244782, 0.6931391805386118, 0.6930491773583609, 0.6928591529055732, 0.6931391805386118, 0.6930191750980527, 0.6931151802186033, 0.693145180558612, 0.6931471805599453, 0.6926971130356336, 0.6930971797265786, 0.6890916952679865, 0.6931291804519437, 0.6924250066981698, 0.6924250066981698, 0.69

C:\Users\siptest\anaconda3\envs\py36\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF
toPlot = np.genfromtxt('C:/Users/siptest/Desktop/NDD/UFIndependenceBinomialNull.csv', delimiter=',')    
ecdf = ECDF(toPlot)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel("p-value")
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.ylabel("fraction of data")
plt.savefig('C:/Users/siptest/Desktop/NDD/UFLinearIndependenceECDF2.png')
plt.show()